<a href="https://colab.research.google.com/github/TWCkaijin/GDGC-Gemini-bootcamp/blob/main/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Function Calling 與 Gemini API & Python SDK 介紹

<td style="text-align: center">
  <a href="https://colab.research.google.com/github/TWCkaijin/GDGC-Gemini-bootcamp/blob/main/function_calling.ipynb#scrollTo=VEqbX8OhE8y9">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
  </a>
</td>

| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt) [Holt Skinner](https://github.com/holtskinner) |

概述
YouTube 影片：AI + 你的程式碼：函式呼叫（Function Calling）

<a href="https://www.youtube.com/watch?v=NbAGbXr4DME&list=PLIivdWyY5sqLvGdVLJZh2EMax97_T-OIB" target="_blank"> <img src="https://img.youtube.com/vi/NbAGbXr4DME/maxresdefault.jpg" alt="AI + 你的程式碼：函式呼叫" width="500"> </a>

</br>

## Gemini
Gemini 是 Google DeepMind 開發的一系列生成式 AI 模型，專為多模態（multimodal）應用場景設計。


</br>

## 從 Gemini 呼叫函式
函式呼叫（Function Calling） 讓開發者可以在程式碼中描述一個函式，然後將該描述傳遞給語言模型進行請求。模型的回應會包含符合該描述的函式名稱以及應該使用的參數。

</br>

## 為什麼要使用函式呼叫？
想像一下，如果你請某人記錄重要資訊，但沒有提供表格或任何格式指引，對方可能會寫出一篇流暢的文章，但如果你需要從中提取特定的姓名、日期或數字，將會非常費力！同樣地，若沒有函式呼叫，想要從生成式文本模型獲得一致的結構化數據會是一大挑戰。你可能得不斷要求模型輸出 JSON 格式，但結果往往不穩定且令人沮喪。

這正是 Gemini 函式呼叫 的優勢所在。與其期待從自由格式的文本回應中拼湊出所需資訊，不如直接定義清楚的函式，並指定具體的參數和資料型別。這些函式定義相當於結構化的指引，能讓 Gemini 以可預測且可用的方式輸出結果。這樣一來，你就不需要再從文字回應中手動解析重要資訊了！

可以把它想像成教 Gemini 如何與你的應用程式對話。
需要從資料庫檢索資訊？定義一個 search_db 函式，並指定搜尋條件作為參數。
想要與天氣 API 整合？建立一個 get_weather 函式，並讓它接收地點作為輸入。
函式呼叫能夠橋接自然語言與結構化數據，讓 AI 更輕鬆地與外部系統互動！

## 任務目標
在本教學中，你將學習如何在 Vertex AI 中使用 Gemini API，並透過 Vertex AI SDK for Python 來使用 Gemini 2.0 Flash (gemini-2.0-flash) 模型進行函式呼叫（Function Calling）。

你將完成以下任務：

- 安裝 _Google Gen AI SDK for Python_
- 在 _Vertex AI_ 中使用 _Gemini API_ 與 _Gemini_ 模型互動：
- 在聊天會話（chat session）中使用 函式呼叫，回答使用者關於 Google Store 產品的問題
- 使用 _Function Calling_ 透過 地圖 API 進行地址地理編碼（geocoding）
- 使用 _Function Calling_ 在 原始日誌數據（raw logging data） 中進行實體擷取（entity extraction）

付費資源
本教學將使用 Google Cloud 中的 計費 功能：

- Vertex AI
請參閱 [Vertex AI](https://cloud.google.com/vertex-ai/pricing)價格 以了解詳細計費資訊，並使用 [費用估算](https://cloud.google.com/products/calculator/) 根據你的預計使用量估算成本。

## Getting Started


### 安裝 Google Gen AI SDK 套件


In [ ]:
%pip install --upgrade --quiet google-genai

### 重啟Colab執行個體

您剛剛安裝了一個套件，為確保他正確載入，我們將重啟執行個體

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 您的執行個體將會重啟，您會在左下角看到警示訊息 ⚠️</b>
</div>


### 驗證Colab環境 (Colab only)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 設定 Google Cloud Platform 專案資訊


要開始使用 Vertex AI，你需要擁有一個 現有的 Google Cloud 專案，並[啟用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

你可以參考[設定專案與開發環境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)來了解更多詳細資訊。

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## 範例程式碼

### 選擇模型

想要了解更多關於Vertax AI 的 AI 模型 和 APIs, see [Google Models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models) and [Model Garden](https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models).

In [ ]:
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}

### 載入套件


In [ ]:
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool
import requests

### 聊天範例：在聊天會話中使用函式呼叫回答使用者關於 Google Store 的問題


在這個範例中，我們將使用 Gemini 的函式呼叫（Function Calling） 來建立一個聊天機器人，該機器人可以回答使用者關於 Google Store 產品的問題。

這樣的應用可以讓 AI 透過結構化的函式呼叫，而不是單純生成自由格式的文字，來提供更準確和一致的資訊。

In [ ]:
get_product_info = FunctionDeclaration(
    name="get_product_info",
    description="Get the stock amount and identifier for a given product",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product_name": {"type": "STRING", "description": "Product name"}
        },
    },
)

get_store_location = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "OBJECT",
        "properties": {"location": {"type": "STRING", "description": "Location"}},
    },
)

place_order = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product": {"type": "STRING", "description": "Product name"},
            "address": {"type": "STRING", "description": "Shipping address"},
        },
    },
)


請注意，函式參數需按照 [OpenAPI JSON Schema](https://spec.openapis.org/oas/v3.0.3#schemawr).
 格式 以 Python 字典（dictionary） 形式指定。

以下是定義工具（tool）的方法，讓 Gemini 模型 可以從 3 個函式 中進行選擇：


In [ ]:
retail_tool = Tool(
    function_declarations=[
        get_product_info,
        get_store_location,
        place_order,
    ],
)

現在，你可以在多輪對話（multi-turn chat session）中初始化 Gemini 模型，並啟用函式呼叫（Function Calling）。

在初始化聊天會話時，可以透過 `tools` 參數一次性指定可用的函式，這樣在後續的請求中就不需要每次重新傳遞這些函式設定。

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        temperature=0,
        tools=[retail_tool],
    ),
)

**注意事項**：
temperature 參數控制生成回應的隨機性：

較低的 temperature（如 0）：適合需要**確定性（deterministic）**的函式，例如傳遞固定格式的參數。
較高的 temperature：適合允許更具創意或多樣性參數值的函式，例如需要較自由輸入的應用場景。


當 temperature = 0 時，模型的輸出大多是確定性的，但仍可能有些許變化。

In [ ]:
prompt = """
Do you have the Pixel 9 in stock?
"""

response = chat.send_message(prompt)
response.function_calls[0]

Gemini API 的回應包含一個結構化的資料物件，其中包括 Gemini 從可用函式中選擇的函式名稱以及對應的參數。

由於本教學的重點是提取函式參數並生成函式呼叫，因此你將使用模擬數據（mock data）來回傳合成回應給 Gemini 模型，而不是直接向 API 伺服器發送請求。（不用擔心！稍後的範例中，我們會進行實際的 API 呼叫。）

In [ ]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

api_response = {"sku": "GA04834-US", "in_stock": "yes"}

In reality, you would execute function calls against an external system or database using your desired client library or REST API.

Now, you can pass the response from the (mock) API request and generate a response for the end user:

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
display(Markdown(response.text))

Next, the user might ask where they can buy a different phone from a nearby store:

In [ ]:
prompt = """
What about the Pixel 9 Pro XL? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.function_calls

Again, you get a response with structured data, but notice that there are two function calls instead of one!

The Gemini model identified that it needs both the `get_product_info` and `get_store_location` functions.
Look closely at the prompt that you used in this conversation turn a few cells up, and you'll notice that the user asked about a product -and- the location of a store.

In cases like this when two or more functions are defined (or when the model predicts multiple function calls to the same function), the Gemini model might sometimes return back-to-back or parallel function call responses within a single conversation turn.

This is expected behavior since the Gemini model predicts which functions it should call at runtime, what order it should call dependent functions in, and which function calls can be parallelized, so that the model can gather enough information to generate a natural language response.

Not to worry! You can repeat the same steps as before and build synthetic payloads that would come from an external APIs:

In [ ]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

product_info_api_response = {"sku": "GA08475-US", "in_stock": "yes"}
store_location_api_response = {
    "store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"
}

Again, you can pass the responses from the (mock) API requests back to the Gemini model:

In [ ]:
response = chat.send_message(
    [
        Part.from_function_response(
            name="get_product_info",
            response={
                "content": product_info_api_response,
            },
        ),
        Part.from_function_response(
            name="get_store_location",
            response={
                "content": store_location_api_response,
            },
        ),
    ]
)
display(Markdown(response.text))

Nice work!

Within a single conversation turn, the Gemini model requested 2 function calls in a row before returning a natural language summary. In reality, you might follow this pattern if you need to make an API call to an inventory management system, and another call to a store location database, customer management system, or document repository.

Finally, the user might ask to order a phone and have it shipped to their address:

In [ ]:
prompt = """
I'd like to order a Pixel 9 Pro XL and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.function_calls

Perfect! The Gemini model extracted the user's selected product and their address. Now you can call an API to place the order:

In [ ]:
# This is where you would make an API request to return the status of their order.
# Use synthetic data to simulate a response payload from an external API.

order_api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

And send the payload from the external API call so that the Gemini API returns a natural language summary to the end user.

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": order_api_response,
        },
    ),
)
display(Markdown(response.text))

And you're done!

You were able to have a multi-turn conversation with the Gemini model using function calls, handling payloads, and generating natural language summaries that incorporated the information from the external systems.

### Address example: Using Automatic Function Calling to geocode addresses with a maps API

In this example, you'll define a function that takes multiple parameters as inputs. Then you'll use automatic function calling in the Gemini API to  make a live API call to convert an address to latitude and longitude coordinates.

Start by writing a Python function:

In [ ]:
def get_location(
    amenity: str | None = None,
    street: str | None = None,
    city: str | None = None,
    county: str | None = None,
    state: str | None = None,
    country: str | None = None,
    postalcode: str | None = None,
) -> list[dict]:
    """
    Get latitude and longitude for a given location.

    Args:
        amenity (str | None): Amenity or Point of interest.
        street (str | None): Street name.
        city (str | None): City name.
        county (str | None): County name.
        state (str | None): State name.
        country (str | None): Country name.
        postalcode (str | None): Postal code.

    Returns:
        list[dict]: A list of dictionaries with the latitude and longitude of the given location.
                    Returns an empty list if the location cannot be determined.
    """
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "amenity": amenity,
        "street": street,
        "city": city,
        "county": county,
        "state": state,
        "country": country,
        "postalcode": postalcode,
        "format": "json",
    }
    # Filter out None values from parameters
    params = {k: v for k, v in params.items() if v is not None}

    try:
        response = requests.get(base_url, params=params, headers={"User-Agent": "none"})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching location data: {e}")
        return []

In this example, you're asking the Gemini model to extract components of the address into specific fields within a structured data object. These fields are then passed to the function you defined and the result is returned to Gemini to make a natural language response.

Send a prompt that includes an address, such as:

In [ ]:
prompt = """
I want to get the coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

Great work! You were able to define a function that the Gemini model used to extract the relevant parameters from the prompt. Then you made a live API call to obtain the coordinates of the specified location.

Here we used the [OpenStreetMap Nominatim API](https://nominatim.openstreetmap.org/ui/search.html) to geocode an address to keep the number of steps in this tutorial to a reasonable number. If you're working with large amounts of address or geolocation data, you can also use the [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding), or any mapping service with an API!

## Conclusions

You have explored the function calling feature through the Google Gen AI Python SDK.

The next step is to enhance your skills by exploring this [documentation page](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling).